In [16]:
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval,
    BertForSequenceClassification)
import torch
import numpy as np

In [2]:
config = AutoConfig.from_pretrained(
    "../output_dirs/classification_test/checkpoint-4500",
)
tokenizer = BertTokenizer.from_pretrained(
    "../output_dirs/classification_test/checkpoint-4500",
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    "../output_dirs/classification_test/checkpoint-4500",
    from_tf=False,
    config=config,
)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
model = model.to(device)

In [10]:
max_seq_length=128
test_input = tokenizer.batch_encode_plus(
    [("test input", None)],
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [12]:
model_inp = dict(test_input)
model_inp['labels'] = [0]
# print(model_inp)
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [13]:
len(model_out)

3

In [14]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([1, 30]), torch.Size([1, 768])]


In [17]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits)

In [18]:
print(predicted_logits)

[[-1.3015641   2.0199366   2.3875945  -4.169917   -1.4164283  -4.2287807
  -1.8702354   0.35857558 -2.724265   -2.6567929  -1.8068223  -2.0170307
  -1.5032878  -2.942142    0.22073346 -0.8266337  -2.2023146   0.89124936
   2.5810018   0.01150025  0.69682723 -0.89142203  0.8963838  -0.4870002
   2.767781   -2.828165   -2.5763001  -0.83430356  0.36374697 -0.0487443 ]]


In [19]:
print(predicted_class)

24
